In [ ]:
%pip install ortools psutil

import os
import csv
import time
import math
import psutil
import resource
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [ ]:
def get_memory_usage_in_kb():
    proc = psutil.Process()
    return proc.memory_info().rss // 1024

In [ ]:
def set_memory_limit_gb(gb_limit):
    soft = hard = gb_limit * 1024**3
    resource.setrlimit(resource.RLIMIT_AS, (soft, hard))

In [ ]:
def parse_tsp_file(path):
    coords = []
    with open(path) as f:
        for line in f:
            if line.strip() == 'NODE_COORD_SECTION':
                break
        for line in f:
            if line.strip() == 'EOF':
                break
            parts = line.split()
            if len(parts) >= 3:
                _, x, y = parts[:3]
                coords.append((float(x), float(y)))
    return coords

In [ ]:
def solve_tsp_with_ortools(coords, ram_limit_gb):
    
    set_memory_limit_gb(ram_limit_gb)
    max_mem_kb = ram_limit_gb * 1024 * 1024
    mem_before = get_memory_usage_in_kb()

    # distance matrix
    n = len(coords)
    dist = [[0]*n for _ in range(n)]
    for i, (xi, yi) in enumerate(coords):
        for j, (xj, yj) in enumerate(coords):
            dist[i][j] = int(math.hypot(xi - xj, yi - yj))

    # OR-Tools routing model
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_idx, to_idx):
        from_node = manager.IndexToNode(from_idx)
        to_node   = manager.IndexToNode(to_idx)
        return dist[from_node][to_node]

    transit_idx = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_idx)

    search_params = pywrapcp.DefaultRoutingSearchParameters()
    search_params.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_params.time_limit.seconds = 300

    start = time.time()
    try:
        solution = routing.SolveWithParameters(search_params)
    except MemoryError:
        return {
            "total_cost": 0.0,
            "solution_time_ms": 0.0,
            "peak_memory_kb": float(get_memory_usage_in_kb())
        }
    end = time.time()

   
    elapsed_ms = (end - start) * 1000
    mem_after  = get_memory_usage_in_kb()
    peak_mem   = max(mem_before, mem_after)

   
    if elapsed_ms > 300_000 or peak_mem > max_mem_kb:
        return {
            "total_cost": 0.0,
            "solution_time_ms": elapsed_ms,
            "peak_memory_kb": peak_mem
        }

    # extract total_cost from the route
    total_cost = 0.0
    if solution:
        idx = routing.Start(0)
        while not routing.IsEnd(idx):
            nxt = solution.Value(routing.NextVar(idx))
            total_cost += routing.GetArcCostForVehicle(idx, nxt, 0)
            idx = nxt

    return {
        "total_cost": float(total_cost),
        "solution_time_ms": elapsed_ms,
        "peak_memory_kb": peak_mem
    }

In [ ]:
def process_tsp_instances(input_dir, output_csv, ram=16, cpu_cores=8):
    with open(output_csv, 'w', newline='') as csvf:
        fieldnames = ['tsp_file', 'ram_gb', 'cpu_cores',
                      'total_cost', 'solution_time_ms', 'peak_memory_kb']
        writer = csv.DictWriter(csvf, fieldnames=fieldnames)
        writer.writeheader()

        count = 0
        for fn in sorted(os.listdir(input_dir)):
            if not fn.endswith('.tsp'):
                continue
            path = os.path.join(input_dir, fn)
            coords = parse_tsp_file(path)
            if not coords:
                continue

            res = solve_tsp_with_ortools(coords, ram)
            writer.writerow({
                'tsp_file': fn,
                'ram_gb': ram,
                'cpu_cores': cpu_cores,
                **res
            })
            csvf.flush()
            count += 1

    print(f"Finished processing {count} instances.")
    print(f"Results written to {output_csv}")

In [ ]:
process_tsp_instances(
    input_dir='tsp',
    output_csv='tsp_cpp/or/or_32_256.csv',
    ram=256,
    cpu_cores=32
)